In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Bidirectional
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers


with open('aggdata.p', 'rb') as handle:
        alldata = pickle.load(handle)
sent = [row[0] for row in alldata]
labels = [row[1] for row in alldata]    
#alldata.close()    
max_features = 10000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(sent)
X = tokenizer.texts_to_sequences(sent)
X = pad_sequences(X)

Y = pd.get_dummies(labels)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


embed_dim = 256
lstm_out = 128

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.5))
model.add(Bidirectional( LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2)))
model.add(Dense(128,activation='tanh'))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer=Adam(lr=0.0001,decay=1e-6),metrics = ['accuracy'])
batch_size = 32
print(model.summary())

checkpointer = ModelCheckpoint(filepath="modeltf12.h5",
                               verbose=1,
                               save_best_only=True,
                               monitor='val_acc',
                               mode='max')
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2, validation_split=0.2, callbacks=[checkpointer,tensorboard])
model.save('modeltf2.10.h5')


Using TensorFlow backend.


(3284, 72) (3284, 2)
(822, 72) (822, 2)


/home/hadoop/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 72, 256)           2560000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 72, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 2,987,394
Trainable params: 2,987,394
Non-trainable params: 0
_________________________________________________________________
None
Train on 2627 samples, validate on 657 samples
Epoch 1/10
 - 27s - loss: 0.6913 - acc: 0.5196 - val_loss: 0.6880 - val_acc: 0.5